# Imports

## Libraries

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
server_ip = os.getenv('SERVER_IP')
port = os.getenv('PORT')

MODEL_URL = f"http://{server_ip}:{port}/v1"

In [2]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device detected")

Torch version: 2.2.0+cu121
CUDA available: True
CUDA device name: NVIDIA GeForce RTX 4070 SUPER


In [3]:
from torchtext.datasets import UDPOS
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

## Data

In [4]:
# Load the train, validation, and test datasets
train_iter, valid_iter, test_iter = UDPOS(root='.data', split=('train', 'valid', 'test'))

In [5]:
# Look at the first few items in the training set
for i, item in enumerate(train_iter):
    print(item)
    if i == 0:  # Print first 5 items
        break

[['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.'], ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']]


## Models

In [6]:
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

pipeline = TokenClassificationPipeline(model=model, tokenizer=tokenizer)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using /root/.cache/torch_exte

In [7]:
outputs = pipeline("A test example")
print(outputs)

[{'entity': 'DT', 'score': 0.9997243, 'index': 1, 'word': 'A', 'start': 0, 'end': 1}, {'entity': 'NN', 'score': 0.9997472, 'index': 2, 'word': 'test', 'start': 2, 'end': 6}, {'entity': 'NN', 'score': 0.99973196, 'index': 3, 'word': 'example', 'start': 7, 'end': 14}]


# Comparison

## Mapping

In [8]:
# Create a dictionary to map Penn Treebank tags to UDPOS tags
penn_to_udpos = {
    'O': 'X',
    '``': 'PUNCT',
    ',': 'PUNCT',
    ':': 'PUNCT',
    '.': 'PUNCT',
    "''": 'PUNCT',
    '$': 'SYM',
    '#': 'SYM',
    'CC': 'CCONJ',
    'CD': 'NUM',
    'DT': 'DET',
    'EX': 'PRON',
    'FW': 'X',
    'IN': 'ADP',
    'JJ': 'ADJ',
    'JJR': 'ADJ',
    'JJS': 'ADJ',
    '-LRB-': 'PUNCT',
    'LS': 'X',
    'MD': 'AUX',
    'NN': 'NOUN',
    'NNP': 'PROPN',
    'NNPS': 'PROPN',
    'NNS': 'NOUN',
    'PDT': 'DET',
    'POS': 'PART',
    'PRP': 'PRON',
    'PRP$': 'PRON',
    'RB': 'ADV',
    'RBR': 'ADV',
    'RBS': 'ADV',
    'RP': 'PART',
    '-RRB-': 'PUNCT',
    'SYM': 'SYM',
    'TO': 'PART',
    'UH': 'INTJ',
    'VB': 'VERB',
    'VBD': 'VERB',
    'VBG': 'VERB',
    'VBN': 'VERB',
    'VBP': 'VERB',
    'VBZ': 'VERB',
    'WDT': 'DET',
    'WP': 'PRON',
    'WP$': 'PRON',
    'WRB' : 'ADV'
}

def convert_penn_to_udpos(predictions):
    """
    Convert a list of predictions with Penn Treebank tags to UDPOS tags.
    
    :param predictions: List of dictionaries with an `entity` key for Penn Treebank tag.
                        Example: [{'entity':'DT', ...}, ...]
    
    :return: List of dictionaries with `entity` key converted to UDPOS tag.
             Example: [{'entity':'DET', ...}, ...]
    """
    
    for prediction in predictions:
        penn_tag = prediction['entity']
        # Map the Penn Treebank tag to UDPOS tag, defaulting to `X` if not found
        prediction['entity'] = penn_to_udpos.get(penn_tag, "X")
    
    return predictions

In [9]:
# Example model output
example_output = [
    {'entity': "DT",  "score": 0.9997243,  "index": 1, "word": "A", "start": 0, "end": 1},
    {'entity': "NN",  "score": 0.9997472,  "index": 2, "word": "test", "start": 2, "end": 6},
    {'entity': "NN",  "score": 0.99973196, "index": 3, "word": "example", "start": 7, "end": 14}
]

# Convert the Penn Treebank tags to UDPOS tags
converted_output = convert_penn_to_udpos(example_output)

# Print the converted output
print(converted_output)


[{'entity': 'DET', 'score': 0.9997243, 'index': 1, 'word': 'A', 'start': 0, 'end': 1}, {'entity': 'NOUN', 'score': 0.9997472, 'index': 2, 'word': 'test', 'start': 2, 'end': 6}, {'entity': 'NOUN', 'score': 0.99973196, 'index': 3, 'word': 'example', 'start': 7, 'end': 14}]


In [10]:
label_map = model.config.id2label

for id, label in label_map.items():
    print(f"ID: {id}, Tag: {label}")

ID: 0, Tag: O
ID: 1, Tag: ``
ID: 2, Tag: ,
ID: 3, Tag: :
ID: 4, Tag: .
ID: 5, Tag: ''
ID: 6, Tag: $
ID: 7, Tag: #
ID: 8, Tag: CC
ID: 9, Tag: CD
ID: 10, Tag: DT
ID: 11, Tag: EX
ID: 12, Tag: FW
ID: 13, Tag: IN
ID: 14, Tag: JJ
ID: 15, Tag: JJR
ID: 16, Tag: JJS
ID: 17, Tag: -LRB-
ID: 18, Tag: LS
ID: 19, Tag: MD
ID: 20, Tag: NN
ID: 21, Tag: NNP
ID: 22, Tag: NNPS
ID: 23, Tag: NNS
ID: 24, Tag: PDT
ID: 25, Tag: POS
ID: 26, Tag: PRP
ID: 27, Tag: PRP$
ID: 28, Tag: RB
ID: 29, Tag: RBR
ID: 30, Tag: RBS
ID: 31, Tag: RP
ID: 32, Tag: -RRB-
ID: 33, Tag: SYM
ID: 34, Tag: TO
ID: 35, Tag: UH
ID: 36, Tag: VB
ID: 37, Tag: VBD
ID: 38, Tag: VBG
ID: 39, Tag: VBN
ID: 40, Tag: VBP
ID: 41, Tag: VBZ
ID: 42, Tag: WDT
ID: 43, Tag: WP
ID: 44, Tag: WP$
ID: 45, Tag: WRB


In [11]:
import requests
import json

# Step 1: List available models
def get_available_models():
    try:
        response = requests.get(f"{MODEL_URL}/models")
        response.raise_for_status()  # Raise an exception for HTTP errors
        models = response.json()
        print("Available Models:", json.dumps(models, indent=4))
        return models.get("models", [])
    except Exception as e:
        print(f"Error fetching models: {e}")
        return []

# Step 2: Generate a response using /v1/chat/completions
def generate_chat_response(model_id, messages):
    try:
        # Prepare the payload for the chat completion request
        payload = {
            "model": model_id,
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 100
        }
        
        # Send POST request
        response = requests.post(f"{MODEL_URL}/chat/completions", json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Parse and display the response
        result = response.json()
        print("Chat Response:", json.dumps(result, indent=4))
        return result
    except Exception as e:
        print(f"Error generating chat response: {e}")
        return None

# Step 3: Generate a response using /v1/completions (optional)
def generate_text_completion(model_id, prompt):
    try:
        # Prepare the payload for the text completion request
        payload = {
            "model": model_id,
            "prompt": prompt,
            "temperature": 0.7,
            "max_tokens": 100
        }
        
        # Send POST request
        response = requests.post(f"{MODEL_URL}/completions", json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Parse and display the response
        result = response.json()
        print("Text Completion Response:", json.dumps(result, indent=4))
        return result
    except Exception as e:
        print(f"Error generating text completion: {e}")
        return None

In [12]:
get_available_models()

Available Models: {
    "data": [
        {
            "id": "mixtral-8x7b-instruct-v0.1",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "text-embedding-nomic-embed-text-v1.5",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "hermes-3-llama-3.1-8b",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "llama-3.2-3b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "llama-3.2-1b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        }
    ],
    "object": "list"
}


[]

In [15]:


# Main function to demonstrate usage
if __name__ == "__main__":
    # Get available models
    models = get_available_models()
    
    if not models:
        print("No models available. Make sure LM Studio is running and a model is loaded.")
    else:
        # Select the first model (or replace with your desired model ID)
        selected_model_id = models[0]["mixtral-8x7b-instruct-v0.1"]
        
        # Example for /v1/chat/completions
        chat_messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello! How are you?"}
        ]
        
        print("\n--- Chat Completion Example ---")
        generate_chat_response(selected_model_id, chat_messages)

Available Models: {
    "data": [
        {
            "id": "mixtral-8x7b-instruct-v0.1",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "text-embedding-nomic-embed-text-v1.5",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "hermes-3-llama-3.1-8b",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "llama-3.2-3b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        },
        {
            "id": "llama-3.2-1b-instruct",
            "object": "model",
            "owned_by": "organization_owner"
        }
    ],
    "object": "list"
}
No models available. Make sure LM Studio is running and a model is loaded.
